## sentence completion

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random

In [1]:
# Step 1: Load and Preprocess Data
file_path = r"Dataset\\en.txt"

# Open the file and read the sentences
with open(file_path, "r", encoding="utf-8") as file:
    sentences = file.readlines()

# Clean the sentences (remove newline characters and extra spaces)
sentences = [sentence.strip() for sentence in sentences]



In [3]:
len(sentences)

1223596

In [9]:
dataset = sentences[:10000]

In [10]:
len(dataset)

10000

In [11]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset)
vocab_size = len(tokenizer.word_index) + 1

In [12]:
# Convert text to sequences
input_sequences = []
for line in dataset:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [13]:
input_sequences

[[191, 223],
 [191, 223, 265],
 [5, 13],
 [5, 13, 3],
 [5, 13, 3, 39],
 [5, 13, 3, 39, 3],
 [5, 13, 3, 39, 3, 324],
 [302, 6],
 [302, 6, 2002],
 [302, 6, 2002, 3612],
 [302, 6, 2002, 3612, 16],
 [302, 6, 2002, 3612, 16, 11],
 [302, 6, 2002, 3612, 16, 11, 6],
 [302, 6, 2002, 3612, 16, 11, 6, 3613],
 [302, 6, 2002, 3612, 16, 11, 6, 3613, 1017],
 [2003, 6],
 [2003, 6, 1438],
 [2003, 6, 1438, 115],
 [1, 2589],
 [1, 2589, 6],
 [1, 2589, 6, 2003],
 [5, 25],
 [5, 25, 19],
 [5, 25, 19, 141],
 [5, 25, 19, 141, 118],
 [57, 22],
 [57, 22, 4],
 [57, 22, 4, 931],
 [57, 22, 4, 931, 14],
 [57, 22, 4, 931, 14, 374],
 [45, 6],
 [45, 6, 139],
 [45, 6, 139, 88],
 [45, 6, 139, 88, 3],
 [45, 6, 139, 88, 3, 266],
 [5, 128],
 [5, 128, 34],
 [5, 128, 34, 81],
 [5, 128, 34, 81, 26],
 [5, 128, 34, 81, 26, 3],
 [5, 128, 34, 81, 26, 3, 101],
 [17, 18],
 [17, 18, 47],
 [17, 18, 47, 1258],
 [17, 18, 47, 1258, 3614],
 [5, 18],
 [5, 18, 8],
 [5, 18, 8, 1],
 [5, 18, 8, 1, 1134],
 [6, 11],
 [6, 11, 4],
 [6, 11, 4, 751]

In [14]:
# Padding sequences
max_seq_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre')
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=vocab_size)


In [15]:

# Step 2: Define LSTM Model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 128, input_length=max_seq_length - 1),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.LSTM(256),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\sbhar\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
# Step 3: Train Model
model.fit(X, y, epochs=1, verbose=1)

2183/2183 ━━━━━━━━━━━━━━━━━━━━ 1499s 687ms/step - accuracy: 0.0519 - loss: 6.7083


In [18]:
# Step 4: Generate Sentence Completions
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
# Example Completion
print(generate_text("where is  ", 2 ))


Where is the meeting


In [30]:
# Example Completion
print(generate_text("Artificial intelligence is transforming ", 3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Artificial intelligence is transforming  the meeting of


### pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# Load WikiText-2 dataset
from torchtext.datasets import WikiText2

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')

# Tokenize the dataset
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))

# Build vocabulary
vocab = Vocab(counter, min_freq=2, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
vocab.set_default_index(vocab['<unk>'])

print(f"Vocabulary Size: {len(vocab)}")


In [ ]:
def text_to_tensor(text_iter, vocab, tokenizer, seq_length=30):
    sequences = []
    for line in text_iter:
        tokens = tokenizer(line)
        indices = [vocab[token] for token in tokens]
        sequences.extend(indices)
    
    # Convert list to PyTorch tensor
    text_tensor = torch.tensor(sequences, dtype=torch.long)
    
    # Create input-output pairs
    inputs = []
    targets = []
    for i in range(len(text_tensor) - seq_length):
        inputs.append(text_tensor[i:i+seq_length])
        targets.append(text_tensor[i+1:i+seq_length+1])

    return torch.stack(inputs), torch.stack(targets)

train_iter = WikiText2(split='train')
X_train, y_train = text_to_tensor(train_iter, vocab, tokenizer)
print(f"Training Data Shape: {X_train.shape}, {y_train.shape}")


In [ ]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.lstm(x, hidden)
        output = self.fc(output)
        return output, hidden

# Define model parameters
vocab_size = len(vocab)
embed_size = 128
hidden_size = 256
num_layers = 2

# Instantiate model
model = LSTMLanguageModel(vocab_size, embed_size, hidden_size, num_layers)


In [ ]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
batch_size = 64
seq_length = 30
hidden = None

for epoch in range(num_epochs):
    total_loss = 0
    for i in range(0, X_train.size(0), batch_size):
        inputs = X_train[i:i+batch_size].to(device)
        targets = y_train[i:i+batch_size].to(device)

        optimizer.zero_grad()
        output, hidden = model(inputs, hidden)

        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(X_train):.4f}")


In [2]:
import random

def generate_text(prompt, model, vocab, tokenizer, seq_length=10, max_words=50):
    model.eval()
    
    words = tokenizer(prompt)
    indices = [vocab[token] for token in words]
    input_seq = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)

    generated_words = words

    hidden = None
    for _ in range(max_words):
        output, hidden = model(input_seq, hidden)
        next_word_index = output.argmax(dim=-1)[:, -1].item()
        next_word = vocab.lookup_token(next_word_index)
        
        generated_words.append(next_word)
        input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_word_index]], dtype=torch.long).to(device)], dim=1)

    return " ".join(generated_words)

# Example Usage
prompt = "The future of AI is"
generated_text = generate_text(prompt, model, vocab, tokenizer)
print("Generated Text:", generated_text)


NameError: name 'vocab' is not defined

# language translate

### torch

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import numpy as np


file_path = r"Dataset\\en.txt"
with open(file_path, "r", encoding="utf-8") as file:
    sentences = file.readlines()
english_sentences = [sentence.strip() for sentence in sentences]


file_path = r"Dataset\\ta.txt"

with open(file_path, "r", encoding="utf-8") as file:
    sentences = file.readlines()
tamil_sentences = [sentence.strip() for sentence in sentences]


# Tokenization function
def tokenize(sentences):
    tokenized = [sentence.lower().replace(",", "").replace("?", "").split() for sentence in sentences]
    return tokenized

# Create vocabularies
def build_vocab(tokenized_sentences):
    counter = Counter()
    for sentence in tokenized_sentences:
        counter.update(sentence)
    vocab = {word: i + 2 for i, (word, _) in enumerate(counter.most_common())}  # Start index from 2
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1
    return vocab

# Convert sentences to numerical sequences
def encode_sentences(tokenized_sentences, vocab):
    return [[vocab.get(word, vocab["<unk>"]) for word in sentence] for sentence in tokenized_sentences]

# Tokenization
eng_tokenized = tokenize(english_sentences)
tam_tokenized = tokenize(tamil_sentences)

# Build vocabulary
eng_vocab = build_vocab(eng_tokenized)
tam_vocab = build_vocab(tam_tokenized)

# Convert text to sequences
eng_sequences = encode_sentences(eng_tokenized, eng_vocab)
tam_sequences = encode_sentences(tam_tokenized, tam_vocab)

# Pad sequences
eng_sequences = pad_sequence([torch.tensor(seq) for seq in eng_sequences], batch_first=True, padding_value=0)
tam_sequences = pad_sequence([torch.tensor(seq) for seq in tam_sequences], batch_first=True, padding_value=0)


In [44]:

# Define the LSTM Encoder-Decoder Model
class Seq2SeqModel(nn.Module):
    def __init__(self, input_dim, output_dim, embed_dim=128, hidden_dim=256):
        super(Seq2SeqModel, self).__init__()
        
        # Encoder
        self.encoder_embedding = nn.Embedding(input_dim, embed_dim, padding_idx=0)
        self.encoder_lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        
        # Decoder
        self.decoder_embedding = nn.Embedding(output_dim, embed_dim, padding_idx=0)
        self.decoder_lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, encoder_input, decoder_input):
        # Encoder
        enc_emb = self.encoder_embedding(encoder_input)
        _, (hidden, cell) = self.encoder_lstm(enc_emb)

        # Decoder
        dec_emb = self.decoder_embedding(decoder_input)
        decoder_output, _ = self.decoder_lstm(dec_emb, (hidden, cell))
        output = self.fc(decoder_output)  # Predict next words
        
        return output


In [45]:

# Model setup
input_dim = len(eng_vocab)
output_dim = len(tam_vocab)
model = Seq2SeqModel(input_dim, output_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prepare training data
X_train = eng_sequences
y_train = tam_sequences

# Shift decoder input (teacher forcing)
decoder_input = y_train[:, :-1]  # Remove last token
decoder_target = y_train[:, 1:]  # Remove first token


In [46]:

# Training loop
num_epochs = 1000
batch_size = 4

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train, decoder_input)
    loss = criterion(outputs.view(-1, output_dim), decoder_target.reshape(-1))
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")



Epoch [0/1000], Loss: 5.9562
Epoch [10/1000], Loss: 5.4402
Epoch [20/1000], Loss: 4.1186
Epoch [30/1000], Loss: 2.4821
Epoch [40/1000], Loss: 1.1143
Epoch [50/1000], Loss: 0.4310
Epoch [60/1000], Loss: 0.1852
Epoch [70/1000], Loss: 0.0945
Epoch [80/1000], Loss: 0.0576
Epoch [90/1000], Loss: 0.0402
Epoch [100/1000], Loss: 0.0308
Epoch [110/1000], Loss: 0.0251
Epoch [120/1000], Loss: 0.0212
Epoch [130/1000], Loss: 0.0183
Epoch [140/1000], Loss: 0.0161
Epoch [150/1000], Loss: 0.0143
Epoch [160/1000], Loss: 0.0129
Epoch [170/1000], Loss: 0.0116
Epoch [180/1000], Loss: 0.0106
Epoch [190/1000], Loss: 0.0097
Epoch [200/1000], Loss: 0.0089
Epoch [210/1000], Loss: 0.0082
Epoch [220/1000], Loss: 0.0076
Epoch [230/1000], Loss: 0.0071
Epoch [240/1000], Loss: 0.0066
Epoch [250/1000], Loss: 0.0062
Epoch [260/1000], Loss: 0.0058
Epoch [270/1000], Loss: 0.0055
Epoch [280/1000], Loss: 0.0052
Epoch [290/1000], Loss: 0.0049
Epoch [300/1000], Loss: 0.0046
Epoch [310/1000], Loss: 0.0044
Epoch [320/1000], L

In [47]:

# Translation function
def translate(sentence):
    model.eval()
    with torch.no_grad():
        sequence = encode_sentences(tokenize([sentence]), eng_vocab)
        sequence = pad_sequence([torch.tensor(seq) for seq in sequence], batch_first=True, padding_value=0)
        
        decoder_input = torch.zeros((1, sequence.shape[1]), dtype=torch.long)  # Empty decoder input
        states_value = model(sequence, decoder_input)
        predicted_seq = torch.argmax(states_value, dim=-1).squeeze(0).tolist()
        
        output_sentence = ' '.join([word for word, index in tam_vocab.items() if index in predicted_seq])
        return output_sentence


In [48]:
# Test Translation
print(translate("I like that movie."))

நான்


### tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np


english_sentences = [
    "Hello, how are you?",
    "I love deep learning.",
    "What is your name?",
    "Where do you live?"
]

tamil_sentences = [
    "வணக்கம், நீங்கள் எப்படி இருக்கிறீர்கள்?",
    "எனக்கு ஆழ்ந்த கற்றல் பிடிக்கும்.",
    "உங்கள் பெயர் என்ன?",
    "நீங்கள் எங்கு வாழ்கிறீர்கள்?"
]

# Tokenize English sentences
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)

# Tokenize Tamil sentences
tam_tokenizer = Tokenizer()
tam_tokenizer.fit_on_texts(tamil_sentences)
tam_vocab_size = len(tam_tokenizer.word_index) + 1
tam_sequences = tam_tokenizer.texts_to_sequences(tamil_sentences)

# Pad sequences to ensure uniform length
max_length = max(max(len(seq) for seq in eng_sequences), max(len(seq) for seq in tam_sequences))
eng_sequences = pad_sequences(eng_sequences, maxlen=max_length, padding='post')
tam_sequences = pad_sequences(tam_sequences, maxlen=max_length, padding='post')

# Split input-output pairs
X_train = eng_sequences
y_train = tam_sequences

# Define Seq2Seq Model (Encoder-Decoder)
embedding_dim = 128
units = 256

# Encoder
encoder_inputs = keras.layers.Input(shape=(max_length,))
enc_emb = keras.layers.Embedding(eng_vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = keras.layers.Input(shape=(max_length,))
dec_emb = keras.layers.Embedding(tam_vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(tam_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile Model
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Shift decoder input (y_train) correctly
decoder_input_data = y_train[:, :-1]  # Remove last token
decoder_target_data = y_train[:, 1:]  # Remove first token

# Ensure shapes match
print("Encoder Input Shape:", X_train.shape)  # (num_samples, max_length)
print("Decoder Input Shape:", decoder_input_data.shape)  # (num_samples, max_length - 1)
print("Decoder Target Shape:", decoder_target_data.shape)  # (num_samples, max_length - 1)

# Fix shape mismatch by adjusting max_length
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_length, padding='post')

# Train Model
model.fit([X_train, decoder_input_data], decoder_target_data, batch_size=64, epochs=100)


# Function to translate English → Tamil
def translate(sentence):
    sequence = eng_tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    states_value = model.predict([sequence, np.zeros((1, max_length))])
    predicted_seq = np.argmax(states_value, axis=-1)[0]
    output_sentence = ' '.join([word for word, index in tam_tokenizer.word_index.items() if index in predicted_seq])
    return output_sentence

Encoder Input Shape: (4, 4)
Decoder Input Shape: (4, 3)
Decoder Target Shape: (4, 3)
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0625 - loss: 2.6386
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4375 - loss: 2.6208
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6250 - loss: 2.6025
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6250 - loss: 2.5831
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6250 - loss: 2.5620
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6250 - loss: 2.5385
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6250 - loss: 2.5119
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6250 - loss: 2.4813
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5625 - loss: 2.4458
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6250 - loss: 2.4040
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6250 - loss

In [55]:
# Test Translation
print(translate("Hello, how are you?"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
நீங்கள் எப்படி இருக்கிறீர்கள்


# Seq data - ICU

### tensorflow

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Load Dataset (Assume it has time-series ICU data)
# Columns: ['patient_id', 'time', 'heart_rate', 'blood_pressure', 'oxygen_level', 'outcome']
df = pd.read_csv("icu_timeseries_data.csv")

# Sort by patient ID and time
df = df.sort_values(by=['patient_id', 'time'])

# Normalize numerical features
scaler = StandardScaler()
df[['heart_rate', 'blood_pressure', 'oxygen_level']] = scaler.fit_transform(df[['heart_rate', 'blood_pressure', 'oxygen_level']])

# Encode categorical target (e.g., mortality outcome: 0 = survived, 1 = deceased)
label_encoder = LabelEncoder()
df['outcome'] = label_encoder.fit_transform(df['outcome'])

# Convert to time-series sequences
sequence_length = 10  # Use past 10 time steps to predict outcome
features = ['heart_rate', 'blood_pressure', 'oxygen_level']
patient_groups = df.groupby('patient_id')

X, y = [], []
for _, group in patient_groups:
    data = group[features].values
    labels = group['outcome'].values
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(labels[i + sequence_length]) 

X = np.array(X)
y = np.array(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM Model
model = Sequential([
    Input(shape=(sequence_length, len(features))),  # Input shape: (10, 3)
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification (mortality prediction)
])

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUC Score: {auc:.4f}")


### pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Simulated ICU dataset (for demonstration purposes)
def generate_synthetic_icu_data(num_samples=1000, time_steps=10, num_features=5):
    X = np.random.rand(num_samples, time_steps, num_features).astype(np.float32)  # Vital signs
    y = np.random.randint(0, 2, size=(num_samples, 1)).astype(np.float32)  # Binary outcome (e.g., survival vs. mortality)
    return X, y

# Prepare data
X, y = generate_synthetic_icu_data()
X_train, y_train = torch.tensor(X[:800]), torch.tensor(y[:800])
X_test, y_test = torch.tensor(X[800:]), torch.tensor(y[800:])
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define LSTM model
class ICU_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.3):
        super(ICU_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Take last time step's output
        return self.sigmoid(output)

# Model parameters
input_dim = 5  # Number of features
hidden_dim = 64
output_dim = 1  # Binary classification
num_layers = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ICU_LSTM(input_dim, hidden_dim, output_dim, num_layers).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=10)

# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            predicted = (y_pred > 0.5).float()
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate the model
evaluate_model(model, test_loader)


Epoch 1/10, Loss: 0.6921
Epoch 2/10, Loss: 0.6891
Epoch 3/10, Loss: 0.6898
Epoch 4/10, Loss: 0.6888
Epoch 5/10, Loss: 0.6891
Epoch 6/10, Loss: 0.6892
Epoch 7/10, Loss: 0.6888
Epoch 8/10, Loss: 0.6891
Epoch 9/10, Loss: 0.6884
Epoch 10/10, Loss: 0.6898
Test Accuracy: 49.50%


# video LSTM

### torch

In [ ]:
import cv2
import os

def extract_frames(video_path, output_dir, fps=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) // fps
    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            cv2.imwrite(f"{output_dir}/frame_{frame_id}.jpg", frame)
            frame_id += 1
        count += 1

    cap.release()
    print(f"Extracted {frame_id} frames.")

# Example usage
extract_frames("example_video.mp4", "frames", fps=10)


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import numpy as np

# Load ResNet model
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))  # Remove final layer
resnet.eval()

# Define transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image_path):
    img = Image.open(image_path).convert("RGB")
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = resnet(img).squeeze().numpy()
    return features

# Extract features from all frames
frame_features = []
for frame_file in sorted(os.listdir("frames")):
    frame_path = os.path.join("frames", frame_file)
    feature_vector = extract_features(frame_path)
    frame_features.append(feature_vector)

frame_features = np.array(frame_features)  # Shape: (num_frames, feature_dim)


In [ ]:
import torch.nn as nn

class VideoLSTM(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, num_layers=2, num_classes=10):
        super(VideoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Take last LSTM output
        return out

# Initialize model
model = VideoLSTM(input_dim=2048, hidden_dim=512, num_layers=2, num_classes=5)


In [ ]:
import torch.optim as optim

# Convert features to tensor
X_train = torch.tensor(frame_features, dtype=torch.float32).unsqueeze(0)  # (batch, seq, feature)
y_train = torch.tensor([0])  # Example label

# Define optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_train)
    predicted_class = torch.argmax(predictions, dim=1).item()
    print(f"Predicted class: {predicted_class}")


### tensorflow

In [ ]:
import cv2
import os

def extract_frames(video_path, output_dir, fps=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) // fps
    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_path = os.path.join(output_dir, f"frame_{frame_id}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_id += 1
        count += 1

    cap.release()
    print(f"Extracted {frame_id} frames.")

# Example usage
extract_frames("example_video.mp4", "frames", fps=10)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

# Load pre-trained ResNet model
resnet = ResNet50(weights="imagenet", include_top=False, pooling="avg")

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = resnet.predict(img_array)
    return features.flatten()

# Extract features from all frames
frame_features = []
for frame_file in sorted(os.listdir("frames")):
    frame_path = os.path.join("frames", frame_file)
    feature_vector = extract_features(frame_path)
    frame_features.append(feature_vector)

frame_features = np.array(frame_features)  # Shape: (num_frames, feature_dim)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
def create_lstm_model(input_shape, num_classes):
    model = Sequential([
        LSTM(512, return_sequences=True, input_shape=input_shape),
        Dropout(0.5),
        LSTM(256),
        Dropout(0.5),
        Dense(128, activation="relu"),
        Dense(num_classes, activation="softmax")
    ])
    return model

# Define input shape (sequence_length, feature_dim)
input_shape = (frame_features.shape[0], frame_features.shape[1])
num_classes = 5  # Example number of categories

# Create model
model = create_lstm_model(input_shape, num_classes)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Convert features to TensorFlow tensors
X_train = np.expand_dims(frame_features, axis=0)  # (batch, seq, feature)
y_train = np.array([[1, 0, 0, 0, 0]])  # Example one-hot encoded label

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=1)


In [ ]:
# Predict video class
y_pred = model.predict(X_train)
predicted_class = np.argmax(y_pred, axis=1)[0]
print(f"Predicted Class: {predicted_class}")
